In [1]:
from langchain import hub
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
# Define a function to format the documents retrieved from the vector store
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [3]:
# Specify the embeddings and directory where the vector store database is located
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
persist_directory = "../chroma_db"

# Load the saved vectorstore
vector_store = Chroma(embedding_function=embedding_function, persist_directory=persist_directory)

# Retrieve and generate using the relevant snippets of the vector_store.
retriever = vector_store.as_retriever()

# Use a standard propmt template to perform simple queries on the loaded vectorstore
# TODO: Need to experiment with different prompt templates
prompt = hub.pull("rlm/rag-prompt")
# [HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

# Instantiate the LLM to use
# llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(model="gpt-4", temperature=1)

# Define the chain
rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(prompt)

C:\Users\jim\AppData\Local\Temp\ipykernel_31936\3784340789.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
C:\Users\jim\AppData\Local\Temp\ipykernel_31936\3784340789.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(embedding_function=embedding_function, persist_directory=persist_directory)
d:

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [9]:
standard_template_text = prompt[0].prompt.template
standard_template_text

"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

In [10]:
# Start asking questions

#question = """The lack of police presence and code enforcement is sending a growing message that these violations are not important. Second item: affordable denver and wanting more information about how the tax will accomplish the goals set by Mayor."""
question = """I want information on getting a compost bim. I have submitted case number: 9578014."""

rag_chain.invoke(question)

'The distribution of compost bins is done by Solid Waste on a district-by-district basis according to a rollout schedule and District 4, where you reside, is scheduled to receive compost bins in 2025. Unfortunately, these schedules are strict and not alterable. This means that you will not be able to obtain a compost bin until then, despite your request for an earlier delivery.'

In [5]:
d4_specific_template_text = """
You are an assistant for question-answering tasks specifically for generating responses to constituent emails.
You work as a senior aide to Councilwoman Diana Romero Campbell, a Denver City Council member for District 4.
You represent the South East Region of the city and county of Denver Colorado USA.
Use the following pieces of retrieved context to help answer the question and generate a response to the constituent. 
If you don't know the answer, just say that you don't know but we will get the information and get back to you. 
Use three to four sentences maximum, keep the answer concise, and be specific to the city and county of Denver.

Question: {question}
Context: {context}
Answer:
"""

In [13]:
# Try with d4_specific_template_text
prompt[0].prompt.template = d4_specific_template_text
rag_chain.invoke(question)

'Thank you for reaching out. I understand your concerns about acquiring a compost bin. As per the rollout schedule, the Department of Transportation and Infrastructure (DOTI) plans to deliver compost bins to District 4 in 2025. Unfortunately, they have strict procedures we must adhere to and our office is unable to expedite this process. Please know that until compost service starts in your neighborhood, you will continue to receive a credit on your invoice.'

In [6]:
import textwrap
textwrap.dedent(d4_specific_template_text)

"\nYou are an assistant for question-answering tasks specifically for generating responses to constituent emails.\nYou work as a senior aide to Councilwoman Diana Romero Campbell, a Denver City Council member for District 4.\nYou represent the South East Region of the city and county of Denver Colorado USA.\nUse the following pieces of retrieved context to help answer the question and generate a response to the constituent. \nIf you don't know the answer, just say that you don't know but we will get the information and get back to you. \nUse three to four sentences maximum, keep the answer concise, and be specific to the city and county of Denver.\n\nQuestion: {question}\nContext: {context}\nAnswer:\n"